# <b>Probabilidade

## Teorema de Bayes

O Teorema de Bayes descreve a probabilidade de um evento, baseada em informações prévias que podem estar relacionadas ao evento.

A fórmula do Teorema de Bayes é:

$$
P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}
$$

Onde:

- $P(A|B)$: probabilidade de $A$ dado que $B$ ocorreu (probabilidade posterior);
- $P(B|A)$: probabilidade de $B$ dado que $A$ ocorreu (verossimilhança);
- $P(A)$: probabilidade de $A$ (probabilidade a priori);
- $P(B)$: probabilidade de $B$ (evidência total).

---

Se houver vários eventos $A_i$, a evidência $P(B)$ pode ser calculada como:

$$
P(B) = \sum_{i=1}^{n} P(B|A_i) \cdot P(A_i)
$$

Essa fórmula permite aplicar o teorema em contextos com múltiplas hipóteses.


In [50]:
def bayes_theorem(likelihood, prior, evidence):
    """
    Calcula a probabilidade posterior usando o Teorema de Bayes.
    
    Parâmetros:
    - likelihood: P(B|A), verossimilhança
    - prior: P(A), probabilidade a priori
    - evidence: P(B), evidência

    Retorna:
    - posterior: P(A|B), probabilidade posterior
    """
    if evidence == 0:
        raise ValueError("A evidência não pode ser zero.")
    
    posterior = (likelihood * prior) / evidence
    return posterior

# Exemplo de uso do Teorema de Bayes:
# 1% da população tem uma doença.
# O teste dá positivo em 99% dos casos com a doença e positivo em 5% dos casos sem a doença.

P_A = 0.01              # Probabilidade de ter a doença
P_B_given_A = 0.99      # Probabilidade do teste ser positivo dado que a pessoa tem a doença
P_B_given_not_A = 0.05  # Probabilidade do teste ser positivo dado que a pessoa NÃO tem a doença
P_not_A = 1 - P_A

# Evidência total
P_B = (P_B_given_A * P_A) + (P_B_given_not_A * P_not_A)

# Aplicando o Teorema de Bayes
posterior = bayes_theorem(P_B_given_A, P_A, P_B)
print(f"Probabilidade de ter a doença dado teste positivo: {posterior:.4f}")

Probabilidade de ter a doença dado teste positivo: 0.1667


## Função de Distribuição Acumulada (FDA ou CDF)

A função de distribuição acumulada (FDA), também chamada de CDF (do inglês *Cumulative Distribution Function*), descreve a **probabilidade de que uma variável aleatória $X$ assuma um valor menor ou igual a um determinado valor $x$**.

$$
F(x) = P(X \leq x)
$$

No caso de dados observados (dados empíricos), a distribuição acumulada empírica é dada por:

$$
\hat{F}(x) = \frac{\text{número de elementos } \leq x}{\text{número total de elementos}}
$$

Essa função é **monotonicamente crescente**, e seu valor está sempre entre 0 e 1.

In [51]:
def distribuicao_acumulada(conjunto_de_dados, x):
    """
    Calcula a função de distribuição acumulada empírica (ECDF) para um valor x.

    Parâmetros:
    - conjunto_de_dados: lista ou array com os dados numéricos
    - x: valor até o qual se deseja calcular a proporção acumulada

    Retorna:
    - F_x: proporção dos valores no conjunto que são menores ou iguais a x

    Fórmula:
    F(x) = número de valores <= x / número total de valores
    """
    n = len(conjunto_de_dados)
    count = sum(1 for i in conjunto_de_dados if i <= x)
    F_x = count / n
    return F_x

# Exemplo de uso
amostra = [45, 56, 67, 70, 75, 80, 83, 88, 90, 95]
x = 75
print(f"A proporção de valores <= {x} é {distribuicao_acumulada(amostra, x)}")

A proporção de valores <= 75 é 0.5


## Função Densidade de Probabilidade (FDP ou PDF)

A função densidade de probabilidade (FDP), conhecida como PDF (*Probability Density Function*), descreve a **probabilidade relativa** de uma variável aleatória contínua assumir um determinado valor.

Para uma variável contínua $X$ com distribuição normal, a PDF é definida por:

$$
f(x) = \frac{1}{\sigma \sqrt{2\pi}} \cdot e^{ -\frac{1}{2} \left( \frac{x - \mu}{\sigma} \right)^2 }
$$

Onde:

- $\mu$ é a média da distribuição
- $\sigma$ é o desvio padrão
- $e$ é a base do logaritmo natural

---

### 📌 Interpretação da PDF

Para variáveis contínuas, a probabilidade de um valor exato ocorrer é **zero**:

$$
P(X = x) = 0
$$

O que realmente interessa é a **probabilidade de um intervalo**, que pode ser obtida calculando a área sob a curva da PDF entre dois pontos $a$ e $b$:

$$
P(a \leq X \leq b) = \int_a^b f(x)\,dx
$$

---

### 📐 Uso da Regra dos Trapézios para resolver a Integral

A ideia da regra dos trapézios é:

- Dividir o intervalo $[a, b]$ em $n$ partes pequenas;
- Calcular a área de cada trapézio sob a curva da PDF;
- Somar todas essas áreas para estimar a integral:

$$
\int_a^b f(x)\,dx \approx \sum_{i=1}^{n} \frac{f(x_i) + f(x_{i+1})}{2} \cdot (x_{i+1} - x_i)
$$

Quanto maior o número de subdivisões, mais precisa será a estimativa da área — e, portanto, da **probabilidade acumulada no intervalo**.

In [52]:
def densidade_normal(x, media=0, desvio=1):
    """
    Calcula a função densidade de probabilidade (PDF) da distribuição normal.

    Parâmetros:
    - x: valor da variável aleatória
    - media: média da distribuição (μ)
    - desvio: desvio padrão da distribuição (σ)

    Retorna:
    - f_x: valor da PDF em x
    """
    pi = 3.141592653589793
    e = 2.718281828459045

    coeficiente = 1 / (desvio * (2 * pi) ** 0.5)
    expoente = -0.5 * ((x - media) / desvio) ** 2
    f_x = coeficiente * (e ** expoente)
    return f_x

def probabilidade_intervalo(a, b, media=0, desvio=1, passos=1000):
    """
    Estima a probabilidade de P(a <= X <= b) para uma distribuição normal,
    usando a regra dos trapézios para integração numérica.

    Parâmetros:
    - a: limite inferior do intervalo
    - b: limite superior do intervalo
    - media: média da distribuição
    - desvio: desvio padrão
    - passos: número de subdivisões (maior = mais preciso)

    Retorna:
    - estimativa da probabilidade no intervalo [a, b]
    """
    largura = (b - a) / passos
    soma = 0
    for i in range(passos):
        x0 = a + i * largura
        x1 = a + (i + 1) * largura
        y0 = densidade_normal(x0, media, desvio)
        y1 = densidade_normal(x1, media, desvio)
        area_trapezio = (y0 + y1) * largura / 2
        soma += area_trapezio
    return soma

# Estimando P(60 <= X <= 80) em uma normal com média 70 e desvio padrão 10
p = probabilidade_intervalo(60, 80, media=70, desvio=10)
print(f"Probabilidade entre 60 e 80: {p:.4f}")

Probabilidade entre 60 e 80: 0.6827
